In [1]:
# import useful libraries

import numpy as np
import itertools
from collections import Counter
from utils import *

In [2]:
# which day is it? 

day = 3

In [3]:
# load puzzle input
    
puzzle_input = load_puzzle_input(day)

print(puzzle_input[:3])

['....411...............838......721.....44..............................................607..................................................', '...&......519..................*..........#.97.........994..............404..............*...&43........440...882.......673.505.............', '.....*......*...892.........971...%....131....*..........*.......515...$.......157.....412.............-.....*.............*............594.']


### Part 1

In [5]:
# code for preparing and testing examples for part 1

example_input_1 = load_example_input(day, 1)
example_output_1 = 4361

In [26]:
def find_symbol_coordinates(puzzle_input):
    non_symbols = [str(x) for x in range(10)] + ['.']
    symbol_coordinates = set()
    for row_id, row in enumerate(puzzle_input):
        for column_id, str_value in enumerate(row):
            if str_value not in non_symbols:
                symbol_coordinates.add((row_id, column_id))
    return symbol_coordinates

def find_valid_part_coordinates(symbol_coordinates):
    valid_part_coordinates = set()
    for row_id, column_id in symbol_coordinates:
        for x_delta in [-1, 0, 1]:
            for y_delta in [-1, 0, 1]:
                valid_part_coordinates.add((row_id + x_delta, column_id + y_delta))
    return valid_part_coordinates

def find_part_numbers(puzzle_input):
    part_numbers = []
    valid_part_coordinates = find_valid_part_coordinates(find_symbol_coordinates(puzzle_input))
    for row_id, row in enumerate(puzzle_input):
        str_int_capture = False
        str_int_value = ''
        start_column_id = 0
        for column_id, str_value in enumerate(row):
            if str_value in [str(x) for x in range(10)] and not str_int_capture:
                str_int_capture = True
                str_int_value = str_value
                start_column_id = column_id
            elif str_value in [str(x) for x in range(10)]:
                str_int_value += str_value
            elif str_int_capture:
                coords_to_check = {
                    (row_id, column_id)
                    for column_id in range(start_column_id, column_id)
                }
                if coords_to_check.intersection(valid_part_coordinates):
                    part_numbers.append(int(str_int_value))
                str_int_capture = False
                str_int_value = ''
        # one more time for the last element of the row
        if str_int_capture:
            coords_to_check = {
                (row_id, column_id)
                for column_id in range(start_column_id, column_id)
            }
            if coords_to_check.intersection(valid_part_coordinates):
                part_numbers.append(int(str_int_value))
    return part_numbers

def part_1_solution(puzzle_input):
    part_numbers = find_part_numbers(puzzle_input)
    return sum(part_numbers)
                

In [27]:
part_1_solution(example_input_1)

4361

In [29]:
part_1_solution(puzzle_input)

527446

### Part 2

In [30]:
example_input_2 = load_example_input(day, 1)
example_output_2 = 467835

In [55]:
# solution here

def find_all_part_coords(puzzle_input):
    """
    {
        key: row number
        value: {
            key: (column start, column end)
            value: int part value
        }
    }
    """
    part_coords = {}
    for row_id, row in enumerate(puzzle_input):
        str_int_capture = False
        str_int_value = ''
        start_column_id = 0
        for column_id, str_value in enumerate(row):
            if str_value in [str(x) for x in range(10)] and not str_int_capture:
                str_int_capture = True
                str_int_value = str_value
                start_column_id = column_id
            elif str_value in [str(x) for x in range(10)]:
                str_int_value += str_value
            elif str_int_capture:
                part_coords.setdefault(row_id, {})
                part_coords[row_id][(start_column_id, column_id - 1)] = int(str_int_value)
                str_int_capture = False
                str_int_value = ''
        # one more time for the last element of the row
        if str_int_capture:
            part_coords.setdefault(row_id, {})
            part_coords[row_id][(start_column_id, column_id)] = int(str_int_value)
    return part_coords

def find_all_gear_coords(puzzle_input):
    gear_coords = []
    for row_id, row in enumerate(puzzle_input):
        gear_coords.extend(
            (row_id, column_id)
            for column_id, str_value in enumerate(row)
            if str_value == '*'
        )
    return gear_coords

def check_if_coord_is_adjacent(start_stop_tuple, single_value):
    """
    Checks whether a single value (+/- 1) is adjacent to any value between start and stop
    """
    return any(
        abs(moving_value - single_value) <= 1
        for moving_value in range(start_stop_tuple[0], start_stop_tuple[1] + 1)
    )
    

def find_gear_ratio(gear_coords, part_coords):
    adjacent_part_values = []
    row_gear, column_gear = gear_coords
    for row_delta in [-1, 0, 1]:
        if row_gear + row_delta in part_coords:
            adjacent_part_values.extend(
                part_coords[row_gear + row_delta][part_column_start_stop_tuple]
                for part_column_start_stop_tuple in part_coords[
                    row_gear + row_delta
                ]
                if check_if_coord_is_adjacent(
                    part_column_start_stop_tuple, column_gear
                )
            )
    if len(adjacent_part_values) == 2:
        return adjacent_part_values[0] * adjacent_part_values[1]
    return 0

def part_2_solution(puzzle_input):
    part_coords = find_all_part_coords(puzzle_input)
    gear_coords = find_all_gear_coords(puzzle_input)
    return sum(find_gear_ratio(gear_coord, part_coords) for gear_coord in gear_coords)

In [50]:
find_all_gear_coords(example_input_2)

[(1, 3), (4, 3), (8, 5)]

In [51]:
find_all_part_coords(example_input_2)

{0: {(0, 2): 467, (5, 7): 114},
 2: {(2, 3): 35, (6, 8): 633},
 4: {(0, 2): 617},
 5: {(7, 8): 58},
 6: {(2, 4): 592},
 7: {(6, 8): 755},
 9: {(1, 3): 664, (5, 7): 598}}

In [52]:
for gear_coords in find_all_gear_coords(example_input_2):
    print(gear_coords)
    print(find_gear_ratio(gear_coords, find_all_part_coords(example_input_2)))
    print()

(1, 3)
[467, 35]
16345

(4, 3)
[617]
0

(8, 5)
[755, 598]
451490



In [56]:
part_2_solution(example_input_2)

467835

In [57]:
part_2_solution(puzzle_input)

73201705